## RSI를 이용한 단순 벡테스트

RSI : 상대적 강도 지수

가격 움직임의 강도를 백분율로 나타내며, 언제 추세가 전활될지 예측 시 유용함

에러 해결

1. matplotlib 버전 변경

    pip install matplotlib==3.2.2
    <br><br>
2. C:\Users\사용자\AppData\Local\Programs\Python\Python39\Lib\site-packages\backtrader\feeds\yahoo.py

    sess = requests.Session()

    sess.headers['User-Agent'] = 'backtrader' # 해당 라인 추가 (line 270)

    for i in range(self.p.retries + 1):  # at least once


In [1]:
from datetime import datetime
import backtrader as bt

class MyStrategy(bt.Strategy):  # bt.Strategy 상속
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close)  # RSI로 사용할 변수
    def next(self):  # 주어진 데이와 지표를 만족시키는 최소 주기마다 자동으로 호출
        if not self.position: # 시장에 참여하지 않고
            if self.rsi < 30: # RSI 30 미만인 경우
                self.order = self.buy() # 매수
        else: # 시장에 참여하고 있고
            if self.rsi > 70: # RSI가 70이상인 경우
                self.order = self.sell() # 매도

cerebro = bt.Cerebro()  # 데이터를 취합하고 벡테스트 or 라이브 트레이딩을 실행한 뒤 결과 출력
cerebro.addstrategy(MyStrategy)
data = bt.feeds.YahooFinanceData(dataname='036570.KS', fromdate=datetime(2019, 1, 1), todate=datetime(2020, 12, 1))
cerebro.adddata(data)
cerebro.broker.setcash(10000000)  # 초기 투자 자금 설정
cerebro.addsizer(bt.sizers.SizerFix, stake=30)  # 주식의 매매 단위

print(f'Initial Portfolio Value : {cerebro.broker.getvalue():,.0f} KRW')
cerebro.run()  # 벡테스트 실행
print(f'Final Portfolio Value   : {cerebro.broker.getvalue():,.0f} KRW')
cerebro.plot()  # 벡테스트 결과 차트

Initial Portfolio Value : 10,000,000 KRW
Final Portfolio Value   : 10,000,000 KRW


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]

## RSI_SMA
21일 단순 이동 평균에 대한 RSI_SMA를 지표로 사용

In [1]:
import backtrader as bt
from datetime import datetime

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None        
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def notify_order(self, order):  # 주문 상태에 변화가 있을 때 마다 자동으로 실행
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:  # 주문 완료 상태이면 상세 주문 정보 표기
            if order.isbuy():
                self.log(f'BUY  : 주가 {order.executed.price:,.0f}, '
                    f'수량 {order.executed.size:,.0f}, '
                    f'수수료 {order.executed.comm:,.0f}, '        
                    f'자산 {cerebro.broker.getvalue():,.0f}') 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else: 
                self.log(f'SELL : 주가 {order.executed.price:,.0f}, '
                    f'수량 {order.executed.size:,.0f}, '
                    f'수수료 {order.executed.comm:,.0f}, '
                    f'자산 {cerebro.broker.getvalue():,.0f}') 
            self.bar_executed = len(self)
        elif order.status in [order.Canceled]:
            self.log('ORDER CANCELD')
        elif order.status in [order.Margin]:
            self.log('ORDER MARGIN')
        elif order.status in [order.Rejected]:
            self.log('ORDER REJECTED')
        self.order = None

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.order = self.buy()
        else:
            if self.rsi > 70:
                self.order = self.sell()

    def log(self, txt, dt=None):  # 텍스트 메세지를 인수로 받아 주문 일자와 함께 출력
        dt = self.datas[0].datetime.date(0)
        print(f'[{dt.isoformat()}] {txt}')

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)
data = bt.feeds.YahooFinanceData(dataname='036570.KS',
    fromdate=datetime(2019, 1, 1), todate=datetime(2021, 12, 1))
cerebro.adddata(data)
cerebro.broker.setcash(10000000)
cerebro.broker.setcommission(commission=0.0014)  # 수수료 0.14%
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)  # 매매 주문을 적용할 주식 수

print(f'Initial Portfolio Value : {cerebro.broker.getvalue():,.0f} KRW')
cerebro.run()
print(f'Final Portfolio Value   : {cerebro.broker.getvalue():,.0f} KRW')
cerebro.plot(style='candlestick')  # 주가 표기

Initial Portfolio Value : 10,000,000 KRW
[2019-02-22] BUY  : 주가 435,498, 수량 21, 수수료 12,600, 자산 9,713,751
[2019-03-29] SELL : 주가 488,464, 수량 -21, 수수료 14,132, 자산 11,067,862
[2019-06-04] BUY  : 주가 458,548, 수량 22, 수수료 14,020, 자산 10,828,914
[2019-08-05] SELL : 주가 510,042, 수량 -22, 수수료 15,595, 자산 12,162,884
[2019-10-07] BUY  : 주가 499,253, 수량 22, 수수료 15,416, 자산 12,190,735
[2020-01-03] SELL : 주가 541,792, 수량 -22, 수수료 16,730, 자산 13,068,974
[2020-03-20] BUY  : 주가 553,678, 수량 22, 수수료 17,368, 자산 13,229,147
[2020-05-26] SELL : 주가 805,259, 수량 -22, 수수료 25,260, 자산 18,663,265
[2020-08-05] BUY  : 주가 832,993, 수량 20, 수수료 23,377, 자산 18,739,161
[2020-12-14] SELL : 주가 887,469, 수량 -20, 수수료 24,906, 자산 19,706,983
[2021-03-30] BUY  : 주가 834,000, 수량 21, 수수료 24,831, 자산 19,916,084
Final Portfolio Value   : 15,407,580 KRW


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]